In [ ]:
import os
from cerebras.cloud.sdk import Cerebras
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain

# Initialize Cerebras client
api_key = os.environ.get("CEREBRAS_API_KEY")
if not api_key:
    raise ValueError("CEREBRAS_API_KEY environment variable is not set.")

client = Cerebras(api_key=api_key)

# Initialize LangChain memory
memory = ConversationBufferMemory()

# Function to interact with Cerebras API
def get_cerebras_response(prompt):
    response = client.chat.completions.create(
        messages=[{"role": "user", "content": prompt}],
        model="llama3.1-8b",
    )
    return response.choices[0].message["content"]

# Create a conversation chain
def chat():
    print("Welcome to the Cerebras Chat! Type 'exit' to end the conversation.")
    while True:
        user_input = input("You: ")
        if user_input.lower() == "/exit":
            print("Goodbye!")
            break

        # Add user input to memory
        memory.chat_memory.add_user_message(user_input)

        # Get the conversation history from memory
        history = memory.load_memory_variables({})["history"]

        # Generate a prompt with conversation history
        prompt = f"{history}\nUser: {user_input}\nAI:"

        # Get response from Cerebras API
        ai_response = get_cerebras_response(prompt)

        # Add AI response to memory
        memory.chat_memory.add_ai_message(ai_response)

        # Print AI response
        print(f"AI: {ai_response}")

# Start the chat
if __name__ == "__main__":
    chat()

In [ ]:
import os
from cerebras.cloud.sdk import Cerebras
from langchain.memory import ConversationBufferMemory

# Initialize Cerebras client
api_key = os.environ.get("CEREBRAS_API_KEY")
if not api_key:
    raise ValueError("CEREBRAS_API_KEY environment variable is not set.")

client = Cerebras(api_key=api_key)

# Initialize LangChain memory
memory = ConversationBufferMemory()

# Function to interact with Cerebras API
def get_cerebras_response(prompt):
    response = client.chat.completions.create(
        messages=[{"role": "user", "content": prompt}],
        model="llama3.1-8b",
    )
    # Debug: Print the entire response to inspect its structure
    print("Response Object:", response)
    
    # Adjust this based on the actual response structure
    # Example: If response.choices[0].message.content is the correct path
    return response.choices[0].message.content

# Create a conversation chain
def chat():
    print("Welcome to the Cerebras Chat! Type '/exit' to end the conversation.")
    while True:
        user_input = input("You: ")
        if user_input.lower() == "/exit":
            print("Goodbye!")
            break

        # Add user input to memory
        memory.chat_memory.add_user_message(user_input)

        # Get the conversation history from memory
        history = memory.load_memory_variables({})["history"]

        # Generate a prompt with conversation history
        prompt = f"{history}\nUser: {user_input}\nAI:"

        # Get response from Cerebras API
        ai_response = get_cerebras_response(prompt)

        # Add AI response to memory
        memory.chat_memory.add_ai_message(ai_response)

        # Print AI response
        print(f"AI: {ai_response}")

# Start the chat
if __name__ == "__main__":
    chat()

In [ ]:
import os
from cerebras.cloud.sdk import Cerebras
from langchain.memory import ConversationBufferMemory
import Prompts  # Import the system prompt

# Initialize Cerebras client
api_key = os.environ.get("CEREBRAS_API_KEY")
if not api_key:
    raise ValueError("CEREBRAS_API_KEY environment variable is not set.")

client = Cerebras(api_key=api_key)

# Initialize LangChain memory
memory = ConversationBufferMemory()

# Add the system prompt to memory at the start
memory.chat_memory.add_ai_message(Prompts.SYSTEM_PROMPT)

# Function to interact with Cerebras API
def get_cerebras_response(prompt):
    response = client.chat.completions.create(
        messages=[
            {"role": "system", "content": Prompts.SYSTEM_PROMPT},  # Use the imported system prompt
            {"role": "user", "content": prompt},
        ],
        model="llama3.1-8b",
    )
    # Debug: Print the entire response to inspect its structure
    print("Response Object:", response)
    
    # Adjust this based on the actual response structure
    # Example: If response.choices[0].message.content is the correct path
    return response.choices[0].message.content

# Create a conversation chain
def chat():
    print("Welcome to the Cerebras Chat! Type '/exit' to end the conversation.")
    while True:
        user_input = input("You: ")
        if user_input.lower() == "/exit":
            print("Goodbye!")
            break

        # Add user input to memory
        memory.chat_memory.add_user_message(user_input)

        # Get the conversation history from memory
        history = memory.load_memory_variables({})["history"]

        # Generate a prompt with conversation history
        prompt = f"{history}\nUser: {user_input}\nAI:"

        # Get response from Cerebras API
        ai_response = get_cerebras_response(prompt)

        # Add AI response to memory
        memory.chat_memory.add_ai_message(ai_response)

        # Print AI response
        print(f"AI: {ai_response}")

# Start the chat
if __name__ == "__main__":
    chat()

In [ ]:
from sentence_transformers import SentenceTransformer

# Load a pre-trained model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Generate embeddings for a list of sentences
sentences = ["Why is fast inference important?", "Another example sentence."]
embeddings = model.encode(sentences)

# Print the embeddings
for i, embedding in enumerate(embeddings):
    print(f"Sentence {i+1} embedding: {embedding}")


In [ ]:
import os
import pinecone
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Pinecone
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from cerebras.cloud.sdk import Cerebras

# =======================
# Configuration
# =======================
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
PINECONE_ENV = os.getenv("PINECONE_ENV", "us-west1-gcp")
PINECONE_INDEX_NAME = "text-vector-store-3f9f8e1e-9d6a-4a5b-8c2d-2f1e3d4b5c6a"  # Your specific index name

CEREBRAS_API_KEY = os.getenv("CEREBRAS_API_KEY")
CEREBRAS_MODEL = "llama3.1-8b"

# Initialize Pinecone
pinecone.init(api_key=PINECONE_API_KEY, environment=PINECONE_ENV)

# Initialize Cerebras client
cerebras_client = Cerebras(api_key=CEREBRAS_API_KEY)

# Initialize embeddings (OpenAI or any other compatible embeddings)
embeddings = OpenAIEmbeddings()

# Load the specific Pinecone index
if PINECONE_INDEX_NAME not in pinecone.list_indexes():
    raise ValueError(f"Pinecone index '{PINECONE_INDEX_NAME}' does not exist.")

vector_store = Pinecone.from_existing_index(PINECONE_INDEX_NAME, embeddings)

# Initialize conversation memory
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

# =======================
# Custom LLM Wrapper for Cerebras
# =======================
class CerebrasLLM:
    def __init__(self, client, model):
        self.client = client
        self.model = model

    def __call__(self, prompt):
        response = self.client.chat.completions.create(
            messages=[{"role": "user", "content": prompt}],
            model=self.model,
        )
        return response.choices[0].message.content

# Initialize Cerebras LLM
cerebras_llm = CerebrasLLM(cerebras_client, CEREBRAS_MODEL)

# =======================
# RAG Pipeline with Conversation Memory
# =======================
qa_chain = ConversationalRetrievalChain.from_llm(
    llm=cerebras_llm,
    retriever=vector_store.as_retriever(),
    memory=memory,
    verbose=True,
)

# =======================
# Chat Function
# =======================
def chat(query):
    try:
        # Get response from RAG pipeline
        result = qa_chain({"question": query})
        return result["answer"]
    except Exception as e:
        return f"Error: {str(e)}"

# =======================
# Example Usage
# =======================
if __name__ == "__main__":
    print("Welcome to the RAG Chat! Type '/exit' to end the conversation.")
    while True:
        user_input = input("You: ")
        if user_input.lower() == "/exit":
            print("Goodbye!")
            break

        response = chat(user_input)
        print(f"AI: {response}")

In [ ]:
import requests
import json

url = "http://127.0.0.1:8000/create-vector-store"
headers = {
    "Content-Type": "application/json"
}
data = {
    "text": "Your text goes here. This could be any text you want to convert into a vector store.",
    "chunk_size": 500,
    "overlap": 50
}

response = requests.post(url, headers=headers, data=json.dumps(data))

print("Status Code:", response.status_code)
print("Response Body:", response.json())

In [ ]:
import requests
import json

url = "http://127.0.0.1:8000/delete-vector-store"
headers = {
    "Content-Type": "application/json"
}
data = {
    "index_name": "vecstored1166eb468c7"
}

response = requests.post(url, headers=headers, data=json.dumps(data))

print("Status Code:", response.status_code)
print("Response Body:", response.json())

In [3]:
import openai

In [ ]:
openai migrate

In [ ]:
from openai import OpenAI

client = OpenAI(api_key="sk-d4199c81edca44709f3eb9c1d2df6f53", base_url="https://api.deepseek.com")

response = client.chat.completions.create(
    model="deepseek-chat",
    messages=[
        {"role": "system", "content": "You are a helpful assistant"},
        {"role": "user", "content": "Hello Tell me about DeepSeek"},
    ],
    stream=False
)

print(response.choices[0].message.content)

In [ ]:
from groq import Groq

client = Groq()
completion = client.chat.completions.create(
    model="deepseek-r1-distill-llama-70b",
    messages=[
        {
            "role": "system",
            "content": "You are a helpful assistant"
        },
        {
            "role": "user",
            "content": "Tell me about LeNet-5 and its architecture" 
        }
    ],
    temperature=0.6,
    top_p=0.95,
    stream=False,
    stop=None,
)

# print(completion.choices[0].message.content)
print(completion.choices[0].message.__dict__.keys())

dict_keys(['content', 'role', 'function_call', 'tool_calls'])
